In [1]:
import arcpy

In [23]:
arcpy.env.workspace = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb"
stations_featureclass = 'SW_stations'
subroutes_fc = 'D:\MyApps\ArcGIS Pro 3.2\greenbelt_gis\Input\MyProject1.gdb\subroutes_featureclass_SW_ExportFeatures1'
greenbelt_fc = 'Greenbelt_1'

# Add fields for greenbelt analysis if they don't exist
fields_to_add = [
    ('gb_direction', 'TEXT'),
    ('gb_distance', 'DOUBLE'),
    ('gb_length', 'DOUBLE')
]

for field_name, field_type in fields_to_add:
    if field_name not in [field.name for field in arcpy.ListFields(stations_featureclass)]:
        arcpy.AddField_management(stations_featureclass, field_name, field_type)

In [19]:
subroutes_fields = [field.name for field in arcpy.ListFields(subroutes_fc)]
if 'upstream_station_id' not in subroutes_fields:
    arcpy.AddField_management(subroutes_fc, 'upstream_station_id', 'TEXT',field_length=9999)
if 'downstream' not in subroutes_fields:
    arcpy.AddField_management(subroutes_fc, 'downstream', 'TEXT',field_length=9999)


In [33]:
def analyze_greenbelt_relations(stations_fc, subroutes_fc, greenbelt_fc):
    # Ensure 'gb_length' field exists in stations feature class
    if 'gb_length' not in [f.name for f in arcpy.ListFields(stations_fc)]:
        arcpy.AddField_management(stations_fc, 'gb_length', 'DOUBLE')

    # Fetch subroute data and store in a dictionary keyed by upstream_station_id
    subroute_data = {}
    with arcpy.da.SearchCursor(subroutes_fc, ["OID@", "SHAPE@", "upstream_station_id"]) as cursor:
        for oid, shape, upstream_id in cursor:
            if upstream_id not in subroute_data:
                subroute_data[upstream_id] = []
            subroute_data[upstream_id].append((oid, shape))

    # Create a layer from the greenbelt feature class for spatial queries
    if arcpy.Exists("greenbelt_lyr"):
        arcpy.Delete_management("greenbelt_lyr")
    arcpy.MakeFeatureLayer_management(greenbelt_fc, "greenbelt_lyr")

    # Near analysis to populate 'NEAR_DIST' in stations feature class
    arcpy.Near_analysis(stations_fc, greenbelt_fc)
    #generate greenbelt length for each subroutes first then match
    # Iterate over each station
    with arcpy.da.UpdateCursor(stations_fc, ["STATION", "SHAPE@", "gb_direction", "gb_distance", "NEAR_DIST", "gb_length"]) as cursor:
        for row in cursor:
            station_id = row[0]

            # Determine if the station is inside the greenbelt
            arcpy.SelectLayerByLocation_management("greenbelt_lyr", "INTERSECT", row[1])
            inside_greenbelt = int(arcpy.GetCount_management("greenbelt_lyr").getOutput(0)) > 0
            row[2] = 'Inside' if inside_greenbelt else 'Outside'

            # Assign NEAR_DIST to 'gb_distance'
            row[3] = row[4]

            # Process relevant subroutes for the current station
            if station_id in subroute_data:
                for subroute_oid, subroute_shape in subroute_data[station_id]:
                    clipped_subroute = arcpy.Clip_analysis(subroute_shape, greenbelt_fc, "in_memory\\clipped")
                    for clipped_row in arcpy.da.SearchCursor("in_memory\\clipped", ["SHAPE@LENGTH"]):
                        clipped_length = clipped_row[0]
                        print(f"Station ID: {station_id}, Subroute OID: {subroute_oid}, Length within Greenbelt: {clipped_length}")
                    arcpy.Delete_management("in_memory\\clipped")

            cursor.updateRow(row)

# Execute the function
analyze_greenbelt_relations(stations_featureclass, subroutes_fc, greenbelt_fc)


In [ ]:
'''past working simple python code'''

def analyze_greenbelt_relations(stations_fc, subroutes_fc, greenbelt_fc):
    # Ensure 'gb_length' field exists in the subroutes feature class
    subroutes_fields = [field.name for field in arcpy.ListFields(subroutes_fc)]
    if 'gb_length' not in subroutes_fields:
        arcpy.AddField_management(subroutes_fc, 'gb_length', 'DOUBLE')

    # Create a layer from the greenbelt feature class for spatial queries
    if arcpy.Exists("greenbelt_lyr"):
        arcpy.Delete_management("greenbelt_lyr")
    arcpy.MakeFeatureLayer_management(greenbelt_fc, "greenbelt_lyr")

    # Near analysis to populate 'NEAR_DIST' for stations
    arcpy.Near_analysis(stations_fc, greenbelt_fc)

    # Iterate over each station
    with arcpy.da.UpdateCursor(stations_fc, ["station_id", "SHAPE@", "gb_direction", "gb_distance", "NEAR_DIST"]) as cursor:
        for row in cursor:
            # [rest of the code remains the same for station analysis]

    # Calculate sub-route lengths within the greenbelt
    with arcpy.da.UpdateCursor(subroutes_fc, ["OID@", "SHAPE@", "gb_length"]) as cursor:
        for row in cursor:
            subroute_id, subroute_geom = row[0], row[1]
            # [rest of the code remains the same for sub-route analysis]

# Execute the function
analyze_greenbelt_relations(stations_featureclass, subroutes_fc, greenbelt_fc)


In [29]:
stations_featureclass

'SW_stations'

In [ ]:
'''archive
def analyze_greenbelt_relations(stations_fc, subroutes_fc, greenbelt_fc):
    # Check if the layer exists and delete it if it does
    if arcpy.Exists("greenbelt_lyr"):
        arcpy.Delete_management("greenbelt_lyr")

    # Ensure 'gb_length' field exists in stations feature class
    stations_fields = [field.name for field in arcpy.ListFields(stations_fc)]
    if 'gb_length' not in stations_fields:
        arcpy.AddField_management(stations_fc, 'gb_length', 'DOUBLE')

    # Create a layer from the greenbelt feature class for spatial queries
    arcpy.MakeFeatureLayer_management(greenbelt_fc, "greenbelt_lyr")

    # Near analysis to populate 'NEAR_DIST' in stations feature class
    arcpy.Near_analysis(stations_fc, greenbelt_fc)

    # Iterate over each station
    with arcpy.da.UpdateCursor(stations_fc, ["STATION", "SHAPE@", "gb_direction", "gb_distance", "NEAR_DIST", "gb_length"]) as cursor:
        for row in cursor:
            station_id = row[0]

            # Calculate the total length of downstream subroutes within greenbelt for this station
            total_gb_length = 0
            subroute_sql = f"upstream_station_id = '{station_id}' AND downstream = 1"
            with arcpy.da.SearchCursor(subroutes_fc, ["SHAPE@"], subroute_sql) as subroutes_cursor:
                for subroute_row in subroutes_cursor:
                    subroute_geom = subroute_row[0]
                    clipped_subroute = arcpy.Clip_analysis(subroute_geom, greenbelt_fc, "in_memory\\clipped")
                    clipped_length = sum([r[0] for r in arcpy.da.SearchCursor("in_memory\\clipped", ["SHAPE@LENGTH"])])
                    total_gb_length += clipped_length
                    arcpy.Delete_management("in_memory\\clipped")

            row[5] = total_gb_length  # Assign the total length of greenbelt intersected subroutes

            # Determine if the station is inside, upstream, or downstream of greenbelt
            arcpy.SelectLayerByLocation_management("greenbelt_lyr", "INTERSECT", row[1])
            if int(arcpy.GetCount_management("greenbelt_lyr").getOutput(0)) > 0:
                row[2] = 'Inside'
            else:
                # Set distance to greenbelt
                row[3] = row[4]  # Assign NEAR_DIST to gb_distance

            cursor.updateRow(row)

# Execute the function
analyze_greenbelt_relations(stations_featureclass, subroutes_fc, greenbelt_fc)
###
'''

In [28]:
#export gis data to csv
import pandas as pd
import os
from arcgis.gis import GIS
from arcgis.features import FeatureLayer

